<a href="https://colab.research.google.com/github/omotayoofere/Music_Information_Retrieval_2/blob/main/Music_Genre_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#For manipulating data
import numpy as np
import pandas as pd

#For Preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, cross_validate
from sklearn import metrics

from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout

#For modelling
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA

#For tuning HyperParameters
from sklearn.model_selection import GridSearchCV

In [ ]:
def load_dataset(src_folder='C:/Users/google cloud/Desktop/genres', dest_filename='data.csv'):
    #opens a new csv file named data.csv
    file = open('data.csv', 'w', newline='')

    with file:#working with created file
        
        writer = csv.writer(file)#writing into data.csv file
        
        writer.writerow(header)#making headers we already declared the title of columns in data.csv

    #creating like a list of genre type and spliting them based on spaces that exists between them
    genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
    #for each of the splitted entity
    for g in genres:
        #pointing to the names of file in a particular directory
        for filename in os.listdir(src_folder + '/' + g):
            #getting a particular file under each genre
            songname = src_folder + '/' + g + '/' + filename
            #loading the particular file using librosa, with duration set to 30secs
            y, sr = librosa.load(songname, mono=True, duration=30)
            #getting chroma stft of a particular file
            chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
            #getting spectral centroid of a particular file
            spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
            #getting spectral bandwidth of a particular file
            spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
            #getting spectral roll-off of a particular file
            rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
            #getting zero crossing rate of a particular file
            zcr = librosa.feature.zero_crossing_rate(y)
            #getting rmse of a particular file
            rmse = librosa.feature.rms(y=y)[0]
            #getting 20-mfcc(s) of a particular file
            mfcc = librosa.feature.mfcc(y=y, sr=sr)
            #appending all the mean of the calculated features for a particular file in a row of data.csv file except the mfcc(s) at this point
            to_append = filename + ' ' + np.mean(chroma_stft) + ' ' + np.mean(rmse) + ' ' + np.mean(spec_cent) + \
            ' ' + np.mean(spec_bw) + ' ' + np.mean(rolloff) + ' ' + np.mean(zcr)
            
            #getting the mean of each of the 20 mfcc(s) of a particular file
            for e in mfcc:
                #appending this set of mfcc mean to the already existing data.csv
                to_append += ' ' + np.mean(e)
            #getting the label of each of the file and attaching it to its respective row
            to_append += ' ' + g
            #opening the created data.csv file and saving all our stuff into it
            file = open(dest_filename, 'a', newline='')
            with file:
                writer = csv.writer(file)
                writer.writerow(to_append.split()) 

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
data_df = pd.read_csv('/content/gdrive/MyDrive/Datasets/features_30_sec.csv')

In [5]:
data_df.head(5)

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.wav,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,blues.00001.wav,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,blues.00002.wav,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,blues.00003.wav,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,blues.00004.wav,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues


In [6]:
#Dropping columns with distict values in all rows
data_df = data_df.drop(['filename','length'], axis=1)

In [7]:
label_cols = data_df.iloc[:,-1]

In [11]:
label_cols.unique()

array(['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz',
       'metal', 'pop', 'reggae', 'rock'], dtype=object)

In [12]:
#getting unique items in output list
unique_labels = np.unique(label_cols)

encoder = LabelEncoder()

cols_encoding = encoder.fit_transform(label_cols)
list_encoding = encoder.fit_transform(unique_labels)

In [13]:
#getting the feature columns
X = data_df.iloc[:,:-1]
y = cols_encoding

In [14]:
X.head(5)

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
0,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,3805.839606,9.015054e+05,...,0.752740,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035
1,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,3550.522098,2.977893e+06,...,0.927998,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282
2,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,3042.260232,7.840345e+05,...,2.451690,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025
3,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,2184.745799,1.493194e+06,...,0.780874,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339
4,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,3579.757627,1.572978e+06,...,-4.520576,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160


In [9]:
scores = []
actual = []
preds = []

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA

In [19]:
pipeline=Pipeline([
    ('scaler', StandardScaler()),
    ("PCA", PCA(n_components=0.99)),
    ('clf', LogisticRegression())
])
pipeline.steps

[('scaler', StandardScaler()),
 ('PCA', PCA(n_components=0.99)),
 ('clf', LogisticRegression())]

In [20]:
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=10)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(800, 57)
(200, 57)
(800,)
(200,)


In [24]:
clfs = []
clfs.append(LogisticRegression())
clfs.append(SVC())
clfs.append(KNeighborsClassifier(n_neighbors=3))
clfs.append(DecisionTreeClassifier())
clfs.append(LogisticRegression())
clfs.append(RandomForestClassifier())

for classifier in clfs:
  pipeline.set_params(clf=classifier)
  scores=cross_validate(pipeline, x_train, y_train)
  print('---------------------------------------------')
  print(str(classifier))
  print('---------------------------------------------')
  for key, values in scores.items():
    print(key, 'mean', values.mean())
    print(key, 'std', values.std())

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


---------------------------------------------
LogisticRegression()
---------------------------------------------
fit_time mean 0.17031311988830566
fit_time std 0.016729476816763372
score_time mean 0.00404973030090332
score_time std 0.00022884529537343576
test_score mean 0.675
test_score std 0.04183300132670376
---------------------------------------------
SVC()
---------------------------------------------
fit_time mean 0.07340264320373535
fit_time std 0.008919272358997067
score_time mean 0.022474288940429688
score_time std 0.004171720016740512
test_score mean 0.70375
test_score std 0.04517327749898162
---------------------------------------------
KNeighborsClassifier(n_neighbors=3)
---------------------------------------------
fit_time mean 0.01229872703552246
fit_time std 0.00021126526308074426
score_time mean 0.012866687774658204
score_time std 0.0005805435787449701
test_score mean 0.6487499999999999
test_score std 0.03147419578003542
---------------------------------------------
De

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


---------------------------------------------
LogisticRegression()
---------------------------------------------
fit_time mean 0.527106237411499
fit_time std 0.04817894216336182
score_time mean 0.011412763595581054
score_time std 0.0011528084957964126
test_score mean 0.675
test_score std 0.04183300132670376
---------------------------------------------
RandomForestClassifier()
---------------------------------------------
fit_time mean 0.6776978492736816
fit_time std 0.29337934914374564
score_time mean 0.028865623474121093
score_time std 0.008658080657806964
test_score mean 0.67125
test_score std 0.020766559657295194


In [ ]:
from sklearn.model_selection import GridSearchCV
pipeline.set_params(clf= SVC())
pipeline.steps

[('scaler', StandardScaler()), ('PCA', PCA(n_components=0.99)), ('clf', SVC())]

In [ ]:
cv_grid = GridSearchCV(pipeline, param_grid = {
    'clf__kernel' : ['linear', 'rbf'],
    'clf__C' : np.linspace(0.1,1.2,12)
})

cv_grid.fit(x_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('PCA', PCA(n_components=0.99)),
                                       ('clf', SVC())]),
             param_grid={'clf__C': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2]),
                         'clf__kernel': ['linear', 'rbf']})

In [ ]:
cv_grid.best_params_

{'clf__C': 1.2, 'clf__kernel': 'rbf'}

In [ ]:
cv_grid.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()), ('PCA', PCA(n_components=0.99)),
                ('clf', SVC(C=1.2))])

In [ ]:
cv_grid.best_score_

0.7162499999999999